# Find local bus routes with same origin / destination

In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import intake
import geopandas as gpd
import pandas as pd

from shapely.geometry import Point

import _utils
from _utils import GCS_FILE_PATH, SELECTED_DATE, COMPILED_CACHED_VIEWS
from shared_utils import geography_utils
from bus_service_utils import gtfs_build

catalog = intake.open_catalog("*.yml")

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
amtrak_routes2 = buffer_around_origin_destination(
    amtrak_routes, buffer_feet = geography_utils.FEET_PER_MI * 5)

In [ ]:
intersect_origin = gpd.sjoin(
    longest_route.set_geometry("origin"), 
    amtrak_routes2[["destination_buffer"]].set_geometry("destination_buffer"), 
    how = "inner",
    predicate = "intersects"
)[["calitp_itp_id", "route_id", "geometry"]].drop_duplicates().set_geometry("geometry")

In [ ]:
intersect_destination = gpd.sjoin(
    intersect_origin.set_geometry("destination"), 
    amtrak_routes2[["destination_buffer"]].set_geometry("destination_buffer"), 
    how = "inner",
    predicate = "intersects"
)[["calitp_itp_id", "route_id", "geometry"]].drop_duplicates().set_geometry("geometry")

In [ ]:
intersect_destination.head()

In [ ]:
test = intersect_destination[(intersect_destination.calitp_itp_id==182) & 
                             (intersect_destination.route_id.str.contains("720"))
                            ]

In [ ]:
test = longest_route[(longest_route.calitp_itp_id==182) &
                     (longest_route.route_id.str.contains("720"))
                    ]

In [ ]:
intersect_origin = gpd.sjoin(
    test.set_geometry("origin"), 
    amtrak_routes2[["route_id", "origin_buffer"]].set_geometry("origin_buffer"), 
    how = "inner",
    predicate = "intersects"
)

intersect_origin

In [ ]:
intersect_destination = gpd.sjoin(
    test.set_geometry("destination"), 
    amtrak_routes2[["route_id", "destination_buffer"]].set_geometry("destination_buffer"), 
    how = "inner",
    predicate = "intersects"
)

intersect_destination

In [ ]:
this_amtrak_route="37329"

intersect_origin[intersect_origin.route_id_right==this_amtrak_route].plot()

In [ ]:
intersect_destination[intersect_destination.route_id_right==this_amtrak_route].plot()

In [ ]:
amtrak_routes2[amtrak_routes2.route_id==this_amtrak_route][
    ["origin_stop_name", "destination_stop_name"]].drop_duplicates()